In [1]:
import math
import numpy as np
import wandb
import random
import torch
import torch_geometric
from torch_geometric.data import Data
import sys
import os
from tqdm import tqdm
import signal
import joblib
import argparse
import json
import os
import subprocess
from torch.utils.data import DataLoader, Dataset, Subset
import help_functions as hf

scripts_path = os.path.abspath(os.path.join('..'))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)
    
import gnn_io as gio
import gnn_architectures as garch

This notebook can be used for debugging - the code is analogous to run_models.py

In [2]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def prepare_data_with_graph_features(data_dict_list, indices_of_datasets_to_use, batch_size, path_to_save_dataloader):
    datalist = [Data(x=d['x'], edge_index=d['edge_index'], pos=d['pos'], y=d['y'], graph_attr=d['graph_attr']) for d in data_dict_list]
    return datalist
    # dataset_only_relevant_dimensions = gio.cut_dimensions(dataset=datalist, indices_of_dimensions_to_keep=indices_of_datasets_to_use)
    # train_set, valid_set, test_set = gio.split_into_subsets(dataset=dataset_only_relevant_dimensions, train_ratio=0.8, val_ratio=0.15, test_ratio=0.05)
    
    # train_set_normalized, x_scaler, pos_scaler = gio.normalize_dataset_create_scaler(dataset_input = train_set, directory_path=path_to_save_dataloader, normalize_y=False, normalize_pos=True)
    # valid_set_normalized = gio.normalize_dataset_with_given_scaler(dataset_input=valid_set, x_scalar_list=x_scaler, pos_scalar=pos_scaler, y_scalar= None,normalize_y=False, normalize_pos=True)
    # test_set_normalized =  gio.normalize_dataset_with_given_scaler(dataset_input=test_set, x_scalar_list=x_scaler, pos_scalar=pos_scaler,y_scalar=None, normalize_y=False, normalize_pos=True)
        
    # train_loader = DataLoader(dataset=train_set_normalized, batch_size=batch_size, shuffle=True, num_workers=4, prefetch_factor=2, pin_memory=True, collate_fn=gio.collate_fn, worker_init_fn=seed_worker)
    # val_loader = DataLoader(dataset=valid_set_normalized, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, collate_fn=gio.collate_fn, worker_init_fn=seed_worker)
    # test_loader = DataLoader(dataset=test_set_normalized, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=gio.collate_fn, worker_init_fn=seed_worker)
    # gio.save_dataloader(test_loader, path_to_save_dataloader + 'test_dl.pt')
    # gio.save_dataloader_params(test_loader, path_to_save_dataloader + 'test_loader_params.json')
    # return train_loader, val_loader
        


In [3]:
# Set parameters here
params = {"project_name": "test",
            "indices_of_datasets_to_use": [0,1,3,4],
            "num_epochs": 1000,
            "batch_size": 8,
            "point_net_conv_layer_structure_local_mlp": [64],
            "point_net_conv_layer_structure_global_mlp": [64],
            "gat_conv_layer_structure": [64, 128],
            "lr": 0.001,
            "gradient_accumulation_steps": 3,
            "in_channels": 6,
            "out_channels": 1,
            "early_stopping_patience": 20,
            "unique_model_description": "my_test",
            "dropout": 0.3,
            "use_dropout": False
        } 

In [4]:
# Create base directory for the run
base_dir = '../../data/' + params['project_name'] + '/'
unique_run_dir = os.path.join(base_dir, params['unique_model_description'])
os.makedirs(unique_run_dir, exist_ok=True)
dataset_path = '../../data/train_data/sim_output_1pm_capacity_reduction_10k_PRELIMINARY.pt'

data_dict_list, model_save_path, path_to_save_dataloader = hf.get_paths(base_dir=base_dir, unique_model_description= params['unique_model_description'], model_save_path= 'trained_model/model.pth', dataset_path=dataset_path)
# train_dl, valid_dl = prepare_data_with_graph_features(data_dict_list=data_dict_list, indices_of_datasets_to_use=params['indices_of_datasets_to_use'], batch_size= params['batch_size'], path_to_save_dataloader= path_to_save_dataloader)
datalist = prepare_data_with_graph_features(data_dict_list=data_dict_list, indices_of_datasets_to_use=params['indices_of_datasets_to_use'], batch_size= params['batch_size'], path_to_save_dataloader= path_to_save_dataloader)

In [7]:
datalist[0].graph_attr

tensor([[1.1676e+03, 3.6210e+03],
        [1.8187e+03, 4.9711e+03],
        [4.8474e+02, 4.4733e+03],
        [7.8859e-01, 1.0573e+03],
        [1.6164e+03, 5.4786e+03],
        [1.0174e+03, 1.2214e+03]])

In [5]:
gpus = hf.get_available_gpus()
best_gpu = hf.select_best_gpu(gpus)
hf.set_cuda_visible_device(best_gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = hf.setup_wandb(params['project_name'], {
    "epochs": params['num_epochs'],
    "batch_size": params['batch_size'],
    "lr": params['lr'],
    "gradient_accumulation_steps": params['gradient_accumulation_steps'],
    "early_stopping_patience": params['early_stopping_patience'],
    "point_net_conv_local_mlp": params['point_net_conv_layer_structure_local_mlp'],
    "point_net_conv_global_mlp": params['point_net_conv_layer_structure_global_mlp'],
    "gat_conv_layer_structure": params['gat_conv_layer_structure'],
    "indices_to_use": params['indices_of_datasets_to_use'],
    "in_channels": params['in_channels'],
    "out_channels": params['out_channels'],
    "dropout": params['dropout'],
    "use_dropout": params['use_dropout']
})

gnn_instance = garch.MyGnn(in_channels=config.in_channels, out_channels=config.out_channels, point_net_conv_layer_structure_local_mlp=config.point_net_conv_local_mlp,
                            point_net_conv_layer_structure_global_mlp=config.point_net_conv_global_mlp,
                            gat_conv_layer_structure=config.gat_conv_layer_structure,
                            dropout=config.dropout, use_dropout=config.use_dropout)
model = gnn_instance.to(device)
loss_fct = torch.nn.MSELoss()

baseline_loss_mean_target = gio.compute_baseline_of_mean_target(dataset=train_dl, loss_fct=loss_fct)
baseline_loss = gio.compute_baseline_of_no_policies(dataset=train_dl, loss_fct=loss_fct)
print("baseline loss mean " + str(baseline_loss_mean_target))
print("baseline loss no  " +str(baseline_loss) )

early_stopping = gio.EarlyStopping(patience=params['early_stopping_patience'], verbose=True)
best_val_loss, best_epoch = garch.train(model=model, 
            config=config, 
            loss_fct=loss_fct,
            optimizer=torch.optim.AdamW(model.parameters(), lr=config.lr, weight_decay=1e-4),
            train_dl=train_dl, 
            valid_dl=valid_dl,
            device=device, 
            early_stopping=early_stopping,
            accumulation_steps=config.gradient_accumulation_steps,
            model_save_path=model_save_path,
            use_gradient_clipping=True,
            lr_scheduler_warmup_steps=20000,
            lr_scheduler_cosine_decay_rate=0.2)
print(f'Best model saved to {model_save_path} with validation loss: {best_val_loss} at epoch {best_epoch}')  

Using GPU 0 with CUDA_VISIBLE_DEVICES=0


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


Model initialized


NameError: name 'train_dl' is not defined